# analysis_cdo_nco_draft2017b.ipynb

## Purpose
Use CDO and NCO to analyse CESM simulation output from project [p17c-marc-comparison](https://github.com/grandey/p17c-marc-comparison).

## Requirements
- Climate Data Operators (CDO)
- NetCDF Operators (NCO)
- CESM output data, post-processed to time-series format, as described in [data_management.org](https://github.com/grandey/p17c-marc-comparison/blob/master/manage_data/data_management.org#syncing-to-local-machine-for-analysis). These data are available via https://doi.org/10.6084/m9.figshare.5687812.

## Author
Benjamin S. Grandey, 2017-2018

In [1]:
! date

Tue Jun 12 16:31:43 +08 2018


In [2]:
from glob import glob
import os
import re
import shutil

## CDO and NCO version information
Useful to record for reproducibility

In [3]:
! cdo --version
! ncks --version

Climate Data Operators version 1.9.1 (http://mpimet.mpg.de/cdo)
Compiled: by root on squall2.local (x86_64-apple-darwin17.2.0) Nov  2 2017 18:28:19
CXX Compiler: /usr/bin/clang++ -std=gnu++11 -pipe -Os -stdlib=libc++ -arch x86_64  -D_THREAD_SAFE -pthread
CXX version : unknown
C Compiler: /usr/bin/clang -pipe -Os -arch x86_64  -D_THREAD_SAFE -pthread
C version : unknown
Features: DATA PTHREADS HDF5 NC4/HDF5 OPeNDAP SZ UDUNITS2 PROJ.4 CURL FFTW3 SSE4_1
Libraries: HDF5/1.10.1 proj/4.93 curl/7.56.1
Filetypes: srv ext ieg grb1 nc1 nc2 nc4 nc4c nc5 
     CDI library version : 1.9.1 of Nov  2 2017 18:27:49
 CGRIBEX library version : 1.9.0 of Sep 29 2017 10:16:02
  NetCDF library version : 4.4.1.1 of Oct  6 2017 14:14:42 $
    HDF5 library version : 1.10.1
 SERVICE library version : 1.4.0 of Nov  2 2017 18:27:47
   EXTRA library version : 1.4.0 of Nov  2 2017 18:27:46
     IEG library version : 1.4.0 of Nov  2 2017 18:27:46
    FILE library version : 1.8.3 of Nov  2 2017 18:27:46

NCO netCDF O

## Directory locations for input and output NetCDF files
The data in the input directory (*in_dir*) are available via Figshare: https://doi.org/10.6084/m9.figshare.5687812.

In [4]:
# Input data directory
#in_dir = os.path.expandvars('$HOME/data/figshare/figshare5687812/')
in_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/output_timeseries/')

# Output data directory
out_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/analysis_cdo_nco_draft2017b/')

## Clean output data directory

In [5]:
for filename in glob('{}/*.nc'.format(out_dir)):
    print('Deleting {}'.format(filename.split('/')[-1]))
    os.remove(filename)
for filename in glob('{}/*.nco'.format(out_dir)):
    print('Deleting {}'.format(filename.split('/')[-1]))
    os.remove(filename)
for filename in glob('{}/*.tmp'.format(out_dir)):
    print('Deleting {}'.format(filename.split('/')[-1]))
    os.remove(filename)
! date

Tue Jun 12 16:31:43 +08 2018


## Calculate annual means of standard 2D atmosphere variables

In [6]:
variable_list = ['OC_LDG',  # MARC pure OC burden, for checking burdens derived using mass-mixing ratios
                 'BURDENSO4', 'BURDENPOM', 'BURDENBC',  # MAM burdens
                 'BURDENSEASALT', 'BURDENDUST',  # MAM burdens cont.
                 'TAU_tot', 'AEROD_v',  # AOD in MARC, MAM
                 'CDNUMC',  # vertically-integrated CDNC
                 'CLDTOT', 'CLDLOW', 'CLDMED', 'CLDHGH',  # cloud fraction
                 'TGCLDLWP', 'TGCLDIWP', 'TGCLDCWP',  # grid-box average water path
                 'PRECC', 'PRECL',  # convective and large-scale precipitation rate (in m/s)
                 'PRECSC', 'PRECSL',  # convective and large-scale snow rate (in m/s)
                 'U10',  # 10m wind speed
                 'FSNTOA', 'FSNTOANOA', 'FSNTOA_d1',  # SW flux at TOA, including clean-sky for MARC, MAM
                 'FSNS', 'FSNSNOA', 'FSNS_d1',  # SW flux at surface, including clean-sky for MAM, MAM
                 'CRF', 'SWCF_d1',  # SW cloud radiative effect in MARC, MAM
                 'LWCF',  # LW cloud radiative effect 
                 'FSNTOACNOA', 'FSNTOAC_d1',  # SW clear-sky clean-sky flux at TOA in MARC, MAM
                ]
for variable in variable_list:
    for model in ['marc', 'mam3', 'mam7']:
        for year in ['1850', '2000']:
            # Check if input file exists
            in_filename = '{}/p17c_{}_{}.cam.h0.{}.nc'.format(in_dir, model, year, variable)
            if os.path.isfile(in_filename):
                print('{}, {}, {}'.format(variable, model, year))
                # Calculate annual means using NCO (with years starting in January)
                annual_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable)
                ! ncra -O --mro -d time,,,12,12 {in_filename} {annual_filename}
                print('  Written {}'.format(annual_filename.split('/')[-1]))
! date 

OC_LDG, marc, 1850
  Written marc_1850_OC_LDG_ANN.nc
OC_LDG, marc, 2000
  Written marc_2000_OC_LDG_ANN.nc
BURDENSO4, mam3, 1850
  Written mam3_1850_BURDENSO4_ANN.nc
BURDENSO4, mam3, 2000
  Written mam3_2000_BURDENSO4_ANN.nc
BURDENSO4, mam7, 1850
  Written mam7_1850_BURDENSO4_ANN.nc
BURDENSO4, mam7, 2000
  Written mam7_2000_BURDENSO4_ANN.nc
BURDENPOM, mam3, 1850
  Written mam3_1850_BURDENPOM_ANN.nc
BURDENPOM, mam3, 2000
  Written mam3_2000_BURDENPOM_ANN.nc
BURDENPOM, mam7, 1850
  Written mam7_1850_BURDENPOM_ANN.nc
BURDENPOM, mam7, 2000
  Written mam7_2000_BURDENPOM_ANN.nc
BURDENBC, mam3, 1850
  Written mam3_1850_BURDENBC_ANN.nc
BURDENBC, mam3, 2000
  Written mam3_2000_BURDENBC_ANN.nc
BURDENBC, mam7, 1850
  Written mam7_1850_BURDENBC_ANN.nc
BURDENBC, mam7, 2000
  Written mam7_2000_BURDENBC_ANN.nc
BURDENSEASALT, mam3, 1850
  Written mam3_1850_BURDENSEASALT_ANN.nc
BURDENSEASALT, mam3, 2000
  Written mam3_2000_BURDENSEASALT_ANN.nc
BURDENSEASALT, mam7, 1850
  Written mam7_1850_BURDENSEASALT_

## Extract data on specific atmosphere model levels

In [7]:
variable_list = ['CCN3', ]
for variable in variable_list:
    for model in ['marc', 'mam3', 'mam7']:
        for year in ['1850', '2000']:
            # Check if input file exists
            in_filename = '{}/p17c_{}_{}.cam.h0.{}.nc'.format(in_dir, model, year, variable)
            if os.path.isfile(in_filename):  # there is no mam7_1850 simulation
                # Loop over model levels of interest
                for level in [30, 24, 19]:  # 30 is bottom level; 24 is ~860hpa; 19 is ~525hPa
                    print('{}, {}, {}, ml{}'.format(variable, model, year, level))
                    # Select data for model level using CDO
                    print('  Selecting data for model level {}'.format(level))
                    level_filename = '{}/temp_{}_{}_{}_ml{}.nc'.format(out_dir, model, year, variable, level)
                    ! cdo -s sellevidx,{level} {in_filename} {level_filename}
                    # Rename variable using NCO
                    print('  Renaming variable to {}_ml{}'.format(variable, level))
                    ! ncrename -v {variable},{variable}_ml{level} {level_filename} >/dev/null 2>/dev/null
                    # Calculate annual means using NCO (with years starting in January)
                    print('  Calculating annual means')
                    annual_filename = '{}/{}_{}_{}_ml{}_ANN.nc'.format(out_dir, model, year, variable, level)
                    ! ncra -O --mro -d time,,,12,12 {level_filename} {annual_filename}
                    print('  Written {}'.format(annual_filename.split('/')[-1]))
                    # Remove temporary file
                    for filename in [level_filename, ]:
                        print('  Removing {}'.format(filename.split('/')[-1]))
                        os.remove(filename)
! date

CCN3, marc, 1850, ml30
  Selecting data for model level 30
  Renaming variable to CCN3_ml30
  Calculating annual means
  Written marc_1850_CCN3_ml30_ANN.nc
  Removing temp_marc_1850_CCN3_ml30.nc
CCN3, marc, 1850, ml24
  Selecting data for model level 24
  Renaming variable to CCN3_ml24
  Calculating annual means
  Written marc_1850_CCN3_ml24_ANN.nc
  Removing temp_marc_1850_CCN3_ml24.nc
CCN3, marc, 1850, ml19
  Selecting data for model level 19
  Renaming variable to CCN3_ml19
  Calculating annual means
  Written marc_1850_CCN3_ml19_ANN.nc
  Removing temp_marc_1850_CCN3_ml19.nc
CCN3, marc, 2000, ml30
  Selecting data for model level 30
  Renaming variable to CCN3_ml30
  Calculating annual means
  Written marc_2000_CCN3_ml30_ANN.nc
  Removing temp_marc_2000_CCN3_ml30.nc
CCN3, marc, 2000, ml24
  Selecting data for model level 24
  Renaming variable to CCN3_ml24
  Calculating annual means
  Written marc_2000_CCN3_ml24_ANN.nc
  Removing temp_marc_2000_CCN3_ml24.nc
CCN3, marc, 2000, ml19
  

## Calculate column loadings from MARC mass mixing ratios
- This is to facilitate comparison with MAM. Although MARC diagnoses some column loadings, the column loading are not available for every aerosol component.
- Regarding the calculation of the mass in each level, the following reference is helpful:  http://nco.sourceforge.net/nco.html#Left-hand-casting. A description of the hybrid vertical coordinate system can be found here: http://www.cesm.ucar.edu/models/atm-cam/docs/usersguide/node25.html.

In [8]:
# Calculate column loadings from mass mixing ratios
for year in ['1850', '2000']:  # loop over emission years
    ! date
    print('year = {}'.format(year))
    # Copy the surface pressure file - necessary for decoding of the hybrid coordinates
    print('  Copying surface pressure file')
    in_filename = '{}/p17c_marc_{}.cam.h0.PS.nc'.format(in_dir, year)
    ps_filename = '{}/temp_marc_{}_PS.nc'.format(out_dir, year)
    shutil.copy2(in_filename, ps_filename)
    # Create file containing NCO commands for calculation of air mass in each model level
    nco_filename = '{}/temp_marc_{}.nco'.format(out_dir, year)
    nco_file = open(nco_filename, 'w')
    nco_file.writelines(['*P_bnds[time,ilev,lat,lon]=hyai*P0+hybi*PS;\n',  # pressures at bounds
                         '*P_delta[time,lev,lat,lon]=P_bnds(:,1:30,:,:)-P_bnds(:,0:29,:,:);\n',  # deltas
                         'mass_air=P_delta/9.807;'])  # mass of air
    nco_file.close()
    # Calculate mass of air in each model level
    print('  Calculating mass of air in each model level')
    mass_air_filename = '{}/temp_marc_{}_mass_air.nc'.format(out_dir, year)
    ! ncap2 -O -v -S {nco_filename} {ps_filename} {mass_air_filename}
    # Loop over mass mixing ratios for different aerosol components
    for aerosol in ['OC', 'MOS', 'OIM', 'BC', 'MBS', 'BIM',
                    'NUC', 'AIT', 'ACC',
                    'SSLT01', 'SSLT02', 'SSLT03', 'SSLT04',  # sea-salt
                    'DST01', 'DST02', 'DST03', 'DST04']:  # dust
        ! date
        print('  aerosol = {}'.format(aerosol))
        # Name of corresponding mass mixing ratio variable
        if aerosol[0:3] in ['SSL', 'DST']:
            mmr_aerosol = aerosol
        else:
            mmr_aerosol = 'm{}'.format(aerosol)
        # Copy the mass mixing ratio file
        print('    Copying the file for {}'.format(mmr_aerosol))
        in_filename = '{}/p17c_marc_{}.cam.h0.{}.nc'.format(in_dir, year, mmr_aerosol)
        mmr_filename = '{}/temp_marc_{}_{}.nc'.format(out_dir, year, mmr_aerosol)
        shutil.copy2(in_filename, mmr_filename)
        # Append the mass of air in each model level
        print('    Appending mass_air')
        ! ncks -A {mass_air_filename} {mmr_filename}
        # Calculate the mass of the aerosol
        print('    Calculating the mass of {} in each model level'.format(aerosol))
        mass_aerosol_filename = '{}/temp_marc_{}_mass_{}.nc'.format(out_dir, year, aerosol)
        ! ncap2 -O -s 'mass_{aerosol}=mass_air*{mmr_aerosol}' {mmr_filename} {mass_aerosol_filename}
        # Sum over levels to calculate column loading (and exclude unwanted variables)
        print('    Summing over levels')
        column_filename = '{}/temp_marc_{}_column_{}.nc'.format(out_dir, year, aerosol)
        ! ncwa -O -x -v mass_air,{mmr_aerosol} -a lev -y sum {mass_aerosol_filename} {column_filename}
        # Rename variable
        print('    Renaming variable to c{}_LDG'.format(aerosol))
        ! ncrename -v mass_{aerosol},c{aerosol}_LDG {column_filename} >/dev/null 2>/dev/null
        # Set units and long_name
        print('    Setting units and long_name')
        ! ncatted -a 'units',c{aerosol}_LDG,o,c,'kg/m2' {column_filename}
        ! ncatted -a 'long_name',c{aerosol}_LDG,o,c,'{aerosol} column loading' {column_filename}
        # Calculate annual means (with years starting in January)
        print('    Calculating annual means')
        annual_filename = '{}/marc_{}_c{}_LDG_ANN.nc'.format(out_dir, year, aerosol)
        ! ncra -O --mro -d time,,,12,12 {column_filename} {annual_filename}
        print('    Written {}'.format(annual_filename.split('/')[-1]))
        # Remove three temporary files
        for filename in [mmr_filename, mass_aerosol_filename, column_filename]:
            print('    Removing {}'.format(filename.split('/')[-1]))
            os.remove(filename)
    # Remove another two temporary files
    for filename in [ps_filename, mass_air_filename, nco_filename]:
            print('  Removing {}'.format(filename.split('/')[-1]))
            os.remove(filename)
! date

Tue Jun 12 16:34:44 +08 2018
year = 1850
  Copying surface pressure file
  Calculating mass of air in each model level
Tue Jun 12 16:35:28 +08 2018
  aerosol = OC
    Copying the file for mOC
    Appending mass_air
    Calculating the mass of OC in each model level
    Summing over levels
    Renaming variable to cOC_LDG
    Setting units and long_name
    Calculating annual means
    Written marc_1850_cOC_LDG_ANN.nc
    Removing temp_marc_1850_mOC.nc
    Removing temp_marc_1850_mass_OC.nc
    Removing temp_marc_1850_column_OC.nc
Tue Jun 12 16:36:17 +08 2018
  aerosol = MOS
    Copying the file for mMOS
    Appending mass_air
    Calculating the mass of MOS in each model level
    Summing over levels
    Renaming variable to cMOS_LDG
    Setting units and long_name
    Calculating annual means
    Written marc_1850_cMOS_LDG_ANN.nc
    Removing temp_marc_1850_mMOS.nc
    Removing temp_marc_1850_mass_MOS.nc
    Removing temp_marc_1850_column_MOS.nc
Tue Jun 12 16:37:06 +08 2018
  aerosol 

    Calculating the mass of MOS in each model level
    Summing over levels
    Renaming variable to cMOS_LDG
    Setting units and long_name
    Calculating annual means
    Written marc_2000_cMOS_LDG_ANN.nc
    Removing temp_marc_2000_mMOS.nc
    Removing temp_marc_2000_mass_MOS.nc
    Removing temp_marc_2000_column_MOS.nc
Tue Jun 12 16:52:56 +08 2018
  aerosol = OIM
    Copying the file for mOIM
    Appending mass_air
    Calculating the mass of OIM in each model level
    Summing over levels
    Renaming variable to cOIM_LDG
    Setting units and long_name
    Calculating annual means
    Written marc_2000_cOIM_LDG_ANN.nc
    Removing temp_marc_2000_mOIM.nc
    Removing temp_marc_2000_mass_OIM.nc
    Removing temp_marc_2000_column_OIM.nc
Tue Jun 12 16:53:46 +08 2018
  aerosol = BC
    Copying the file for mBC
    Appending mass_air
    Calculating the mass of BC in each model level
    Summing over levels
    Renaming variable to cBC_LDG
    Setting units and long_name
    Calculat

In [9]:
# Compare cOC_LDG (calculated above) with standard OC_LDG
for year in ['1850', '2000']:
    # Input files
    in_filename_1 = '{}/marc_{}_OC_LDG_ANN.nc'.format(out_dir, year)
    in_filename_2 = '{}/marc_{}_cOC_LDG_ANN.nc'.format(out_dir, year)
    # Rename cOC_LDG to OC_LDG to enable comparison
    temp_filename = '{}/temp_marc_{}_cOC_LDG_ANN.nc'.format(out_dir, year)
    ! ncrename -O -v cOC_LDG,OC_LDG {in_filename_2} {temp_filename} >/dev/null 2>/dev/null
    # Compare using CDO
    ! cdo diffv {in_filename_1} {temp_filename}
    # Remove temporary file
    for filename in [temp_filename, ]:
            print('  Removing {}'.format(filename.split('/')[-1]))
            os.remove(filename)
! date

               Date     Time   Level Gridsize    Miss    Diff : S Z  Max_Absdiff Max_Reldiff : Parameter name
    16 : 0001-07-16 22:00:00       0    13824       0   13824 : F F   1.8680e-08   0.0043520 : OC_LDG     
    28 : 0002-07-16 22:00:00       0    13824       0   13824 : F F   1.7920e-08   0.0033875 : OC_LDG     
    40 : 0003-07-16 22:00:00       0    13824       0   13824 : F F   2.2142e-08   0.0034263 : OC_LDG     
    52 : 0004-07-16 22:00:00       0    13824       0   13824 : F F   2.0513e-08   0.0037291 : OC_LDG     
    64 : 0005-07-16 22:00:00       0    13824       0   13824 : F F   2.2773e-08   0.0037233 : OC_LDG     
    76 : 0006-07-16 22:00:00       0    13824       0   13824 : F F   2.0662e-08   0.0039365 : OC_LDG     
    88 : 0007-07-16 22:00:00       0    13824       0   13824 : F F   1.9976e-08   0.0037408 : OC_LDG     
   100 : 0008-07-16 22:00:00       0    13824       0   13824 : F F   2.2041e-08   0.0038573 : OC_LDG     
   112 : 0009-07-16 22:00:00      

In results above, the maximum relative difference is less than 1%. This shows that the calculation of the column loadings is working as intended.

## Derive additional atmosphere variables

In [10]:
# Derived variables that require *two* input variables
derived_variable_dict = {'ctOC_LDG=cOC_LDG+cOIM_LDG': ['marc',],  # total OC loading
                         'ctBC_LDG=cBC_LDG+cBIM_LDG': ['marc',],  # total BC loading
                         'cPRECT=PRECC+PRECL': ['marc', 'mam3', 'mam7'],  # total precipitation rate
                         'cPRECST=PRECSC+PRECSL': ['marc', 'mam3', 'mam7'],  # total snow rate
                         'cFNTOA=FSNTOA+LWCF': ['marc', 'mam3', 'mam7'],  # net (SW + LW) radiative effect
                         'cDRE=FSNTOA-FSNTOANOA': ['marc',],  # direct radiative effect at TOA
                         'cDRE=FSNTOA-FSNTOA_d1': ['mam3', 'mam7'],
                         'cDREsurf=FSNS-FSNSNOA': ['marc',],  # direct radiative effect at surface
                         'cDREsurf=FSNS-FSNS_d1': ['mam3', 'mam7'],
                         'cAAA=cDRE-cDREsurf': ['marc', 'mam3', 'mam7'],  # absorption by aerosols in atmosphere
                        }
for derived_variable, model_list in derived_variable_dict.items():
    for model in model_list:
        year_list = ['1850', '2000']
        for year in year_list:
            print('{}, {}, {}'.format(derived_variable, model, year))
            # Merge input files
            variable_list = re.split('\=|\+|\-', derived_variable)
            in_filename_1 = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable_list[1])
            in_filename_2 = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable_list[2])
            merge_filename = '{}/temp_{}_{}_merge_ANN.nc'.format(out_dir, model, year)
            ! cdo -s merge {in_filename_1} {in_filename_2} {merge_filename} >/dev/null 2>/dev/null
            # Calculate derived variable
            out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable_list[0])
            ! cdo -s expr,'{derived_variable}' {merge_filename} {out_filename}
            if os.path.isfile(out_filename):
                print('  Written {}'.format(out_filename.split('/')[-1]))
            # Remove temporary file
            for filename in [merge_filename, ]:
                print('  Removing {}'.format(filename.split('/')[-1]))
                os.remove(filename)
! date

ctOC_LDG=cOC_LDG+cOIM_LDG, marc, 1850
  Written marc_1850_ctOC_LDG_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
ctOC_LDG=cOC_LDG+cOIM_LDG, marc, 2000
  Written marc_2000_ctOC_LDG_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
ctBC_LDG=cBC_LDG+cBIM_LDG, marc, 1850
  Written marc_1850_ctBC_LDG_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
ctBC_LDG=cBC_LDG+cBIM_LDG, marc, 2000
  Written marc_2000_ctBC_LDG_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
cPRECT=PRECC+PRECL, marc, 1850
  Written marc_1850_cPRECT_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
cPRECT=PRECC+PRECL, marc, 2000
  Written marc_2000_cPRECT_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
cPRECT=PRECC+PRECL, mam3, 1850
  Written mam3_1850_cPRECT_ANN.nc
  Removing temp_mam3_1850_merge_ANN.nc
cPRECT=PRECC+PRECL, mam3, 2000
  Written mam3_2000_cPRECT_ANN.nc
  Removing temp_mam3_2000_merge_ANN.nc
cPRECT=PRECC+PRECL, mam7, 1850
  Written mam7_1850_cPRECT_ANN.nc
  Removing temp_mam7_1850_merge_ANN.nc
cPRECT=PRECC+PRECL, mam7, 20

In [11]:
# ctSSLT_LDG and ctDST_LDG require *four* input variables
for sslt_or_dst in ['SSLT', 'DST']:  # sea-salt or dust
    variable_list = ['ct{}_LDG'.format(sslt_or_dst), ]
    for size_bin in ['01', '02', '03', '04']:
        variable_list.append('c{}{}_LDG'.format(sslt_or_dst, size_bin))
    derived_variable = '{}={}+{}+{}+{}'.format(*variable_list)
    model = 'marc'  # MARC only
    year_list = ['1850', '2000']
    for year in year_list:
        print('{}, {}, {}'.format(derived_variable, model, year))
        # Merge input files
        in_filename_list = []
        for variable in variable_list[1:]:
            in_filename_list.append('{}/{}_{}_{}_ANN.nc'.format(out_dir, model,
                                                                year, variable))
        merge_filename = '{}/temp_{}_{}_merge_ANN.nc'.format(out_dir, model, year)
        ! cdo -s merge {in_filename_list[0]} {in_filename_list[1]} \
            {in_filename_list[2]} {in_filename_list[3]} {merge_filename} >/dev/null 2>/dev/null
        # Calculate derived variable
        out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable_list[0])
        ! cdo -s expr,'{derived_variable}' {merge_filename} {out_filename}
        if os.path.isfile(out_filename):
            print('  Written {}'.format(out_filename.split('/')[-1]))
        # Remove temporary file
        for filename in [merge_filename, ]:
            print('  Removing {}'.format(filename.split('/')[-1]))
            os.remove(filename)
! date

ctSSLT_LDG=cSSLT01_LDG+cSSLT02_LDG+cSSLT03_LDG+cSSLT04_LDG, marc, 1850
  Written marc_1850_ctSSLT_LDG_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
ctSSLT_LDG=cSSLT01_LDG+cSSLT02_LDG+cSSLT03_LDG+cSSLT04_LDG, marc, 2000
  Written marc_2000_ctSSLT_LDG_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
ctDST_LDG=cDST01_LDG+cDST02_LDG+cDST03_LDG+cDST04_LDG, marc, 1850
  Written marc_1850_ctDST_LDG_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
ctDST_LDG=cDST01_LDG+cDST02_LDG+cDST03_LDG+cDST04_LDG, marc, 2000
  Written marc_2000_ctDST_LDG_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
Tue Jun 12 17:06:21 +08 2018


In [14]:
# ctSUL_LDG requires *seven* input variables
derived_variable_dict = {'ctSUL_LDG=cACC_LDG+cAIT_LDG+cNUC_LDG+cMOS_LDG+cMBS_LDG-cOIM_LDG-cBIM_LDG':
                         ['marc',],}  # total SO4 loading
for derived_variable, model_list in derived_variable_dict.items():
    for model in model_list:
        year_list = ['1850', '2000']
        for year in year_list:
            print('{}, {}, {}'.format(derived_variable, model, year))
            # Merge input files
            variable_list = re.split('\=|\+|\-', derived_variable)
            in_filename_list = []
            for variable in variable_list[1:]:
                in_filename_list.append('{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable))
            merge_filename = '{}/temp_{}_{}_merge_ANN.nc'.format(out_dir, model, year)
            ! cdo -s merge {in_filename_list[0]} {in_filename_list[1]} {in_filename_list[2]} \
               {in_filename_list[3]} {in_filename_list[4]} {in_filename_list[5]} \
               {in_filename_list[6]} {merge_filename} >/dev/null 2>/dev/null
            # Calculate derived variable
            out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable_list[0])
            ! cdo -s expr,'{derived_variable}' {merge_filename} {out_filename}
            if os.path.isfile(out_filename):
                print('  Written {}'.format(out_filename.split('/')[-1]))
            # Remove temporary file
            for filename in [merge_filename, ]:
                print('  Removing {}'.format(filename.split('/')[-1]))
                os.remove(filename)
! date

ctSUL_LDG=cACC_LDG+cAIT_LDG+cNUC_LDG+cMOS_LDG+cMBS_LDG-cOIM_LDG-cBIM_LDG, marc, 1850
  Written marc_1850_ctSUL_LDG_ANN.nc
  Removing temp_marc_1850_merge_ANN.nc
ctSUL_LDG=cACC_LDG+cAIT_LDG+cNUC_LDG+cMOS_LDG+cMBS_LDG-cOIM_LDG-cBIM_LDG, marc, 2000
  Written marc_2000_ctSUL_LDG_ANN.nc
  Removing temp_marc_2000_merge_ANN.nc
Tue Jun 12 17:08:47 +08 2018


## Calculate annual means of land variables

In [15]:
variable_list = ['FSNO',  # fraction of ground covered by snow
                 'SNOBCMSL',  # mass of BC in top layer of snow
                 'BCDEP'  # total BC deposition (dry+wet) from atmosphere
                ]
for variable in variable_list:
    for model in ['marc', 'mam3', 'mam7']:
        for year in ['1850', '2000']:
            # Check if input file exists
            in_filename = '{}/p17c_{}_{}.clm2.h0.{}.nc'.format(in_dir, model, year, variable)
            if os.path.isfile(in_filename):
                print('{}, {}, {}'.format(variable, model, year))
                # Calculate annual means using NCO (with years starting in January)
                temp_filename = '{}/temp_{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable)
                ! ncra -O --mro -d time,,,12,12 {in_filename} {temp_filename}
                # Replace missing values with zero
                out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable)
                ! cdo -s setmisstoc,0 {temp_filename} {out_filename}
                print('  Written {}'.format(out_filename.split('/')[-1]))
                # Remove temporary file
                for filename in [temp_filename, ]:
                    print('  Removing {}'.format(filename.split('/')[-1]))
                    os.remove(filename)
! date

FSNO, marc, 1850
  Written marc_1850_FSNO_ANN.nc
  Removing temp_marc_1850_FSNO_ANN.nc
FSNO, marc, 2000
  Written marc_2000_FSNO_ANN.nc
  Removing temp_marc_2000_FSNO_ANN.nc
FSNO, mam3, 1850
  Written mam3_1850_FSNO_ANN.nc
  Removing temp_mam3_1850_FSNO_ANN.nc
FSNO, mam3, 2000
  Written mam3_2000_FSNO_ANN.nc
  Removing temp_mam3_2000_FSNO_ANN.nc
FSNO, mam7, 1850
  Written mam7_1850_FSNO_ANN.nc
  Removing temp_mam7_1850_FSNO_ANN.nc
FSNO, mam7, 2000
  Written mam7_2000_FSNO_ANN.nc
  Removing temp_mam7_2000_FSNO_ANN.nc
SNOBCMSL, marc, 1850
  Written marc_1850_SNOBCMSL_ANN.nc
  Removing temp_marc_1850_SNOBCMSL_ANN.nc
SNOBCMSL, marc, 2000
  Written marc_2000_SNOBCMSL_ANN.nc
  Removing temp_marc_2000_SNOBCMSL_ANN.nc
SNOBCMSL, mam3, 1850
  Written mam3_1850_SNOBCMSL_ANN.nc
  Removing temp_mam3_1850_SNOBCMSL_ANN.nc
SNOBCMSL, mam3, 2000
  Written mam3_2000_SNOBCMSL_ANN.nc
  Removing temp_mam3_2000_SNOBCMSL_ANN.nc
SNOBCMSL, mam7, 1850
  Written mam7_1850_SNOBCMSL_ANN.nc
  Removing temp_mam7_1850

## Calculate annual means of sea ice variables and remap to lonlat grid
Note: the data are initially on a curvilinear grid.

In [16]:
# Create grid file
# Note: this grid is only approximately the same as the land grid
grid_filename = '{}/grid.txt'.format(in_dir)
grid_file = open(grid_filename, 'w')
grid_file.writelines(['gridtype = lonlat\n',
                      'xsize    = 144\n',
                      'ysize    = 96\n',
                      'xfirst   = 0\n',
                      'xinc     = 2.5\n',
                      'yfirst   = -90\n',
                      'yinc     = 1.89473724\n'])
grid_file.close()
print('Written {}'.format(grid_filename.split('/')[-1]))
!date

Written grid.txt
Tue Jun 12 17:09:17 +08 2018


In [17]:
variable_list = ['fs',  # grid-cell mean snow fraction over ice
                ]
for variable in variable_list:
    for model in ['marc', 'mam3', 'mam7']:
        for year in ['1850', '2000']:
            # Check if input file exists
            in_filename = '{}/p17c_{}_{}.cice.h.{}.nc'.format(in_dir, model, year, variable)
            if os.path.isfile(in_filename):
                print('{}, {}, {}'.format(variable, model, year))
                # Calculate annual means using NCO (with years starting in January)
                annual_filename = '{}/temp_{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable)
                ! ncra -O --mro -d time,,,12,12 {in_filename} {annual_filename}
                # Apply distance weighted remapping to new grid using CDO
                regrid_filename = '{}/temp_{}_{}_{}_ANN_regrid.nc'.format(out_dir, model, year, variable)
                ! cdo -s remapdis,{grid_filename} {annual_filename} {regrid_filename}
                # Replace missing values with zero
                out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, variable)
                ! cdo -s setmisstoc,0 {regrid_filename} {out_filename}
                print('  Written {}'.format(out_filename.split('/')[-1]))
                # Remove temporary file
                for filename in [annual_filename, regrid_filename]:
                    print('  Removing {}'.format(filename.split('/')[-1]))
                    os.remove(filename)
! date

fs, marc, 1850
  Written marc_1850_fs_ANN.nc
  Removing temp_marc_1850_fs_ANN.nc
  Removing temp_marc_1850_fs_ANN_regrid.nc
fs, marc, 2000
  Written marc_2000_fs_ANN.nc
  Removing temp_marc_2000_fs_ANN.nc
  Removing temp_marc_2000_fs_ANN_regrid.nc
fs, mam3, 1850
  Written mam3_1850_fs_ANN.nc
  Removing temp_mam3_1850_fs_ANN.nc
  Removing temp_mam3_1850_fs_ANN_regrid.nc
fs, mam3, 2000
  Written mam3_2000_fs_ANN.nc
  Removing temp_mam3_2000_fs_ANN.nc
  Removing temp_mam3_2000_fs_ANN_regrid.nc
fs, mam7, 1850
  Written mam7_1850_fs_ANN.nc
  Removing temp_mam7_1850_fs_ANN.nc
  Removing temp_mam7_1850_fs_ANN_regrid.nc
fs, mam7, 2000
  Written mam7_2000_fs_ANN.nc
  Removing temp_mam7_2000_fs_ANN.nc
  Removing temp_mam7_2000_fs_ANN_regrid.nc
Tue Jun 12 17:11:54 +08 2018


## Combine snow cover over land and sea ice

In [18]:
for model in ['marc', 'mam3', 'mam7']:
    for year in ['1850', '2000']:
        # Check if input files exists
        lnd_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, 'FSNO')
        ice_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, 'fs')
        if os.path.isfile(lnd_filename) and os.path.isfile(lnd_filename):
            print('{}, {}'.format(model, year))
            # Remap land data to identical grid as was used for ice
            lnd_regrid_filename = '{}/temp_{}_{}_{}_ANN_regrid.nc'.format(out_dir, model, year, 'FSNO')
            ! cdo -s remapnn,{grid_filename} {lnd_filename} {lnd_regrid_filename}
            # Merge land and ice data into one file
            merge_filename = '{}/temp_{}_{}_merge_ANN.nc'.format(out_dir, model, year)
            ! cdo -s merge {lnd_regrid_filename} {ice_filename} {merge_filename}
            # Combine snow cover over land and ice, weighting by land fraction
            out_filename = '{}/{}_{}_{}_ANN.nc'.format(out_dir, model, year, 'cSnowCover')
            derivation_str = '_oceanfrac=1-landfrac;cSnowCover=landfrac*FSNO+_oceanfrac*fs'
            ! cdo -s expr,'{derivation_str}' {merge_filename} {out_filename}
            print('  Written {}'.format(out_filename.split('/')[-1]))
            # Remove temporary files
            for filename in [lnd_regrid_filename, merge_filename]:
                print('  Removing {}'.format(filename.split('/')[-1]))
                os.remove(filename)
! date

marc, 1850
  Written marc_1850_cSnowCover_ANN.nc
  Removing temp_marc_1850_FSNO_ANN_regrid.nc
  Removing temp_marc_1850_merge_ANN.nc
marc, 2000
  Written marc_2000_cSnowCover_ANN.nc
  Removing temp_marc_2000_FSNO_ANN_regrid.nc
  Removing temp_marc_2000_merge_ANN.nc
mam3, 1850
  Written mam3_1850_cSnowCover_ANN.nc
  Removing temp_mam3_1850_FSNO_ANN_regrid.nc
  Removing temp_mam3_1850_merge_ANN.nc
mam3, 2000
  Written mam3_2000_cSnowCover_ANN.nc
  Removing temp_mam3_2000_FSNO_ANN_regrid.nc
  Removing temp_mam3_2000_merge_ANN.nc
mam7, 1850
  Written mam7_1850_cSnowCover_ANN.nc
  Removing temp_mam7_1850_FSNO_ANN_regrid.nc
  Removing temp_mam7_1850_merge_ANN.nc
mam7, 2000
  Written mam7_2000_cSnowCover_ANN.nc
  Removing temp_mam7_2000_FSNO_ANN_regrid.nc
  Removing temp_mam7_2000_merge_ANN.nc
Tue Jun 12 17:11:58 +08 2018


In [19]:
! date

Tue Jun 12 17:11:58 +08 2018
